In [1]:
# Importing the packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import LinearSVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import ConfusionMatrixDisplay, f1_score, make_scorer, confusion_matrix, mean_squared_error, mean_absolute_error

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier

from scipy.stats import randint, uniform

In [2]:
# Reading the Data for Inspection
raw_data = pd.read_csv('../data/raw/car_insurance_claim.csv')

In [3]:
raw_data.head()

,ID,KIDSDRIV,BIRTH,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,...,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,63581743,0,16MAR39,60.0,0,11.0,"$67,349",No,$0,z_No,...,Minivan,yes,"$4,461",2,No,3,$0,18.0,0,Highly Urban/ Urban
1,132761049,0,21JAN56,43.0,0,11.0,"$91,449",No,"$257,252",z_No,...,Minivan,yes,$0,0,No,0,$0,1.0,0,Highly Urban/ Urban
2,921317019,0,18NOV51,48.0,0,11.0,"$52,881",No,$0,z_No,...,Van,yes,$0,0,No,2,$0,10.0,0,Highly Urban/ Urban
3,727598473,0,05MAR64,35.0,1,10.0,"$16,039",No,"$124,191",Yes,...,z_SUV,no,"$38,690",2,No,3,$0,10.0,0,Highly Urban/ Urban
4,450221861,0,05JUN48,51.0,0,14.0,NaN,No,"$306,251",Yes,...,Minivan,yes,$0,0,No,0,$0,6.0,0,Highly Urban/ Urban


In [4]:
pd.set_option('display.max_columns', None)
raw_data.head()

,ID,KIDSDRIV,BIRTH,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,GENDER,EDUCATION,OCCUPATION,TRAVTIME,CAR_USE,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CLM_AMT,CAR_AGE,CLAIM_FLAG,URBANICITY
0,63581743,0,16MAR39,60.0,0,11.0,"$67,349",No,$0,z_No,M,PhD,Professional,14,Private,"$14,230",11,Minivan,yes,"$4,461",2,No,3,$0,18.0,0,Highly Urban/ Urban
1,132761049,0,21JAN56,43.0,0,11.0,"$91,449",No,"$257,252",z_No,M,z_High School,z_Blue Collar,22,Commercial,"$14,940",1,Minivan,yes,$0,0,No,0,$0,1.0,0,Highly Urban/ Urban
2,921317019,0,18NOV51,48.0,0,11.0,"$52,881",No,$0,z_No,M,Bachelors,Manager,26,Private,"$21,970",1,Van,yes,$0,0,No,2,$0,10.0,0,Highly Urban/ Urban
3,727598473,0,05MAR64,35.0,1,10.0,"$16,039",No,"$124,191",Yes,z_F,z_High School,Clerical,5,Private,"$4,010",4,z_SUV,no,"$38,690",2,No,3,$0,10.0,0,Highly Urban/ Urban
4,450221861,0,05JUN48,51.0,0,14.0,NaN,No,"$306,251",Yes,M,<High School,z_Blue Collar,32,Private,"$15,440",7,Minivan,yes,$0,0,No,0,$0,6.0,0,Highly Urban/ Urban


In [5]:
# Cleaning the Data
data_df = raw_data.copy() # Creating a copy to work with

In [6]:
# Update column names to make them more descriptive and easier to work with.
col_names = {
    'KIDSDRIV': 'num_young_drivers',
    'BIRTH': 'date_of_birth',
    'AGE': 'age',
    'HOMEKIDS': 'num_of_children',
    'YOJ': 'years_job_held_for',
    'INCOME': 'income',
    'PARENT1': 'single_parent',
    'HOME_VAL': 'value_of_home',
    'MSTATUS': 'married',
    'GENDER': 'gender',
    'EDUCATION': 'highest_education',
    'OCCUPATION': 'occupation',
    'TRAVTIME': 'commute_dist',
    'CAR_USE': 'type_of_use',
    'BLUEBOOK': 'vehicle_value',
    'TIF': 'policy_tenure',
    'CAR_TYPE': 'vehicle_type',
    'RED_CAR': 'red_vehicle',
    'OLDCLAIM': '5_year_total_claims_value',
    'CLM_FREQ': '5_year_num_of_claims',
    'REVOKED': 'licence_revoked',
    'MVR_PTS': 'license_points',
    'CLM_AMT': 'new_claim_value',
    'CAR_AGE': 'vehicle_age',
    'CLAIM_FLAG': 'is_claim',
    'URBANICITY': 'address_type'
}

# Update column names
data_df = data_df.rename(columns=col_names)
data_df.head()

,ID,num_young_drivers,date_of_birth,age,num_of_children,years_job_held_for,income,single_parent,value_of_home,married,gender,highest_education,occupation,commute_dist,type_of_use,vehicle_value,policy_tenure,vehicle_type,red_vehicle,5_year_total_claims_value,5_year_num_of_claims,licence_revoked,license_points,new_claim_value,vehicle_age,is_claim,address_type
0,63581743,0,16MAR39,60.0,0,11.0,"$67,349",No,$0,z_No,M,PhD,Professional,14,Private,"$14,230",11,Minivan,yes,"$4,461",2,No,3,$0,18.0,0,Highly Urban/ Urban
1,132761049,0,21JAN56,43.0,0,11.0,"$91,449",No,"$257,252",z_No,M,z_High School,z_Blue Collar,22,Commercial,"$14,940",1,Minivan,yes,$0,0,No,0,$0,1.0,0,Highly Urban/ Urban
2,921317019,0,18NOV51,48.0,0,11.0,"$52,881",No,$0,z_No,M,Bachelors,Manager,26,Private,"$21,970",1,Van,yes,$0,0,No,2,$0,10.0,0,Highly Urban/ Urban
3,727598473,0,05MAR64,35.0,1,10.0,"$16,039",No,"$124,191",Yes,z_F,z_High School,Clerical,5,Private,"$4,010",4,z_SUV,no,"$38,690",2,No,3,$0,10.0,0,Highly Urban/ Urban
4,450221861,0,05JUN48,51.0,0,14.0,NaN,No,"$306,251",Yes,M,<High School,z_Blue Collar,32,Private,"$15,440",7,Minivan,yes,$0,0,No,0,$0,6.0,0,Highly Urban/ Urban


In [7]:
# Read the general info of each feature
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10302 entries, 0 to 10301
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         10302 non-null  int64  
 1   num_young_drivers          10302 non-null  int64  
 2   date_of_birth              10302 non-null  object 
 3   age                        10295 non-null  float64
 4   num_of_children            10302 non-null  int64  
 5   years_job_held_for         9754 non-null   float64
 6   income                     9732 non-null   object 
 7   single_parent              10302 non-null  object 
 8   value_of_home              9727 non-null   object 
 9   married                    10302 non-null  object 
 10  gender                     10302 non-null  object 
 11  highest_education          10302 non-null  object 
 12  occupation                 9637 non-null   object 
 13  commute_dist               10302 non-null  int

In [8]:
# Check for duplicated rows
print(data_df.duplicated().sum())

1


In [9]:
# Since there is only a single duplicated row, I'll drop it.
data_df.drop_duplicates(inplace=True)

In [10]:
print(data_df.duplicated().sum())

0


In [11]:
# Investigating some features that have extra characters in their values.
for col in ['single_parent', 'married', 'gender', 'highest_education', 'occupation', 'type_of_use',
            'vehicle_type', 'red_vehicle', 'licence_revoked', 'address_type']:
    print(f"\n--- {col} ---")
    print(data_df[col].value_counts(dropna=False))


--- single_parent ---
single_parent
No     8958
Yes    1343
Name: count, dtype: int64

--- married ---
married
Yes     6187
z_No    4114
Name: count, dtype: int64

--- gender ---
gender
z_F    5545
M      4756
Name: count, dtype: int64

--- highest_education ---
highest_education
z_High School    2952
Bachelors        2822
Masters          2078
<High School     1515
PhD               934
Name: count, dtype: int64

--- occupation ---
occupation
z_Blue Collar    2288
Clerical         1590
Professional     1407
Manager          1257
Lawyer           1031
Student           899
Home Maker        843
NaN               665
Doctor            321
Name: count, dtype: int64

--- type_of_use ---
type_of_use
Private       6512
Commercial    3789
Name: count, dtype: int64

--- vehicle_type ---
vehicle_type
z_SUV          2883
Minivan        2693
Pickup         1772
Sports Car     1179
Van             921
Panel Truck     853
Name: count, dtype: int64

--- red_vehicle ---
red_vehicle
no     7326
yes 

In [12]:
# Cleaning the features with values that start with 'z_'
cols_to_clean = ['married', 'gender', 'highest_education', 'occupation', 'vehicle_type', 'address_type']

data_df[cols_to_clean] = data_df[cols_to_clean].apply(lambda x: x.str.replace(r'^z_', '', regex=True))

In [13]:
for col in ['single_parent', 'married', 'gender', 'highest_education', 'occupation', 'type_of_use',
            'vehicle_type', 'red_vehicle', 'licence_revoked', 'address_type']:
    print(f"\n--- {col} ---")
    print(data_df[col].value_counts(dropna=False))


--- single_parent ---
single_parent
No     8958
Yes    1343
Name: count, dtype: int64

--- married ---
married
Yes    6187
No     4114
Name: count, dtype: int64

--- gender ---
gender
F    5545
M    4756
Name: count, dtype: int64

--- highest_education ---
highest_education
High School     2952
Bachelors       2822
Masters         2078
<High School    1515
PhD              934
Name: count, dtype: int64

--- occupation ---
occupation
Blue Collar     2288
Clerical        1590
Professional    1407
Manager         1257
Lawyer          1031
Student          899
Home Maker       843
NaN              665
Doctor           321
Name: count, dtype: int64

--- type_of_use ---
type_of_use
Private       6512
Commercial    3789
Name: count, dtype: int64

--- vehicle_type ---
vehicle_type
SUV            2883
Minivan        2693
Pickup         1772
Sports Car     1179
Van             921
Panel Truck     853
Name: count, dtype: int64

--- red_vehicle ---
red_vehicle
no     7326
yes    2975
Name: count,

In [14]:
# Removing the $ sign in front of the currency values.
currency_cols = ['income', 'value_of_home', 'vehicle_value', '5_year_total_claims_value', 'new_claim_value']

data_df[currency_cols] = (data_df[currency_cols]
                            .apply(lambda x: x.str.replace(r'^\$', '', regex=True)) # removing leaing $
                            .apply(lambda x: x.str.replace(',', ''))                # removing commas
                            .astype('Int64')                                        # convering to nullable integer type
                         )

In [15]:
for col in ['income', 'value_of_home', 'vehicle_value', '5_year_total_claims_value', 'new_claim_value']:
    print(f"\n--- {col} ---")
    print(data_df[col].value_counts(dropna=False))


--- income ---
income
0         797
<NA>      570
61790       5
26840       4
64916       4
         ... 
43112       1
164669      1
107204      1
53235       1
16039       1
Name: count, Length: 8152, dtype: Int64

--- value_of_home ---
value_of_home
0         2908
<NA>       575
176219       3
225043       3
99103        3
          ... 
386273       1
332591       1
170611       1
197017       1
334777       1
Name: count, Length: 6335, dtype: Int64

--- vehicle_value ---
vehicle_value
1500     207
6200      47
6000      42
5800      39
5600      38
        ... 
13560      1
41790      1
27840      1
36980      1
35280      1
Name: count, Length: 2985, dtype: Int64

--- 5_year_total_claims_value ---
5_year_total_claims_value
0        6291
1391        4
4188        4
1310        4
4448        4
         ... 
35324       1
5208        1
7332        1
3436        1
9550        1
Name: count, Length: 3545, dtype: Int64

--- new_claim_value ---
new_claim_value
0       7555
4363       4

In [16]:
# Dropping the unnecessary features ['ID', 'date_of_birth']
data_df = data_df.drop(['ID', 'date_of_birth'], axis=1)
data_df

,num_young_drivers,age,num_of_children,years_job_held_for,income,single_parent,value_of_home,married,gender,highest_education,occupation,commute_dist,type_of_use,vehicle_value,policy_tenure,vehicle_type,red_vehicle,5_year_total_claims_value,5_year_num_of_claims,licence_revoked,license_points,new_claim_value,vehicle_age,is_claim,address_type
0,0,60.0,0,11.0,67349,No,0,No,M,PhD,Professional,14,Private,14230,11,Minivan,yes,4461,2,No,3,0,18.0,0,Highly Urban/ Urban
1,0,43.0,0,11.0,91449,No,257252,No,M,High School,Blue Collar,22,Commercial,14940,1,Minivan,yes,0,0,No,0,0,1.0,0,Highly Urban/ Urban
2,0,48.0,0,11.0,52881,No,0,No,M,Bachelors,Manager,26,Private,21970,1,Van,yes,0,0,No,2,0,10.0,0,Highly Urban/ Urban
3,0,35.0,1,10.0,16039,No,124191,Yes,F,High School,Clerical,5,Private,4010,4,SUV,no,38690,2,No,3,0,10.0,0,Highly Urban/ Urban
4,0,51.0,0,14.0,<NA>,No,306251,Yes,M,<High School,Blue Collar,32,Private,15440,7,Minivan,yes,0,0,No,0,0,6.0,0,Highly Urban/ Urban
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10297,1,45.0,2,9.0,164669,No,386273,Yes,M,PhD,Manager,21,Private,13270,15,Minivan,no,0,0,No,2,0,17.0,0,Highly Urban/ Urban
10298,0,46.0,0,9.0,107204,No,332591,Yes,M,Masters,NaN,36,Commercial,24490,6,Panel Truck,no,0,0,No,0,0,1.0,0,Highly Urban/ Urban
10299,0,48.0,0,15.0,39837,No,170611,Yes,F,<High School,Blue Collar,12,Private,13820,7,SUV,no,0,0,No,0,0,1.0,0,Highly Urban/ Urban
10300,0,50.0,0,7.0,43445,No,149248,Yes,F,Bachelors,Home Maker,36,Private,22550,6,Minivan,no,0,0,No,0,0,11.0,0,Highly Urban/ Urban


In [17]:
# Save cleaned dataset to the processed data folder
data_df.to_csv('../data/processed/car_insurance_claim_processed.csv', index=False)